In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

/home/dro/.local/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("train_dataset_train.csv")
test = pd.read_csv('test_dataset_test.csv')

/tmp/ipykernel_615702/398589533.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_dataset_train.csv")
/tmp/ipykernel_615702/398589533.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('test_dataset_test.csv')


In [3]:
df["is_in_yandex"] = pd.Categorical(df["is_in_yandex"])
df["is_in_yandex"].astype('category').cat.codes
df["is_in_yandex"] = df["is_in_yandex"].cat.codes

In [4]:
test["is_in_yandex"] = pd.Categorical(test["is_in_yandex"])
test["is_in_yandex"].astype('category').cat.codes
test["is_in_yandex"] = test["is_in_yandex"].cat.codes

In [5]:
df["is_return"] = pd.Categorical(df["is_return"])
df["is_return"].astype('category').cat.codes
df["is_return"] = df["is_return"].cat.codes

In [6]:
test["is_return"] = pd.Categorical(test["is_return"])
test["is_return"].astype('category').cat.codes
test["is_return"] = test["is_return"].cat.codes

Обьединим список не нужных строк с списком строк типа object

In [7]:
col_obj = df.select_dtypes(include=['object']).columns.values
col_obj = list(set(col_obj) ^ set(["id", "label"]))

In [8]:
col_obj

['name_mfi',
 'type',
 'index_oper',
 'is_privatecategory',
 'label',
 'oper_type + oper_attr',
 'id']

In [9]:
X = df.drop(col_obj, axis = 1)
y = df[["label"]]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

## Обучение модели

In [11]:
import lightgbm as lgb

In [35]:
clf = lgb.LGBMClassifier(num_leaves=8023, random_state=42)

In [36]:
clf.fit(X_train, y_train, eval_set=(X_test, y_test))

/home/dro/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dro/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.106725
[2]	valid_0's binary_logloss: 0.100913
[3]	valid_0's binary_logloss: 0.0967008
[4]	valid_0's binary_logloss: 0.0934574
[5]	valid_0's binary_logloss: 0.0908481
[6]	valid_0's binary_logloss: 0.0886759
[7]	valid_0's binary_logloss: 0.0868675
[8]	valid_0's binary_logloss: 0.08536
[9]	valid_0's binary_logloss: 0.0840366
[10]	valid_0's binary_logloss: 0.0829165
[11]	valid_0's binary_logloss: 0.0819493
[12]	valid_0's binary_logloss: 0.0810898
[13]	valid_0's binary_logloss: 0.0803226
[14]	valid_0's binary_logloss: 0.079681
[15]	valid_0's binary_logloss: 0.0791489
[16]	valid_0's binary_logloss: 0.0786146
[17]	valid_0's binary_logloss: 0.0781884
[18]	valid_0's binary_logloss: 0.0778248
[19]	valid_0's binary_logloss: 0.0775037
[20]	valid_0's binary_logloss: 0.0771882
[21]	valid_0's binary_logloss: 0.0769307
[22]	valid_0's binary_logloss: 0.0767312
[23]	valid_0's binary_logloss: 0.0765295
[24]	valid_0's binary_logloss: 0.0763985
[25]	valid_0's binary_logloss:

LGBMClassifier(num_leaves=8023, random_state=42)

## Оценка точности

In [37]:
pred = clf.predict_proba(X_test)[:, 1] > 0.009

In [38]:
from sklearn.metrics import recall_score

score = recall_score(y_test, pred, average = "macro" )

In [39]:
print("Recall", score)

Recall 0.8805939941513985


In [40]:
ss = pd.read_csv('sample_solution.csv')

In [41]:
ss['label'] = (clf.predict_proba(test[X_test.columns])[:, 1] > 0.009).astype(int)


In [42]:
ss.to_csv('solut.csv', index=False)

In [43]:
ss['label'].mean()

0.2049975